## Logistic Regression 


In [9]:
import sys
sys.path.append('../')
import glob2 as glob
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from sklearn.preprocessing import OneHotEncoder

pd.options.display.max_columns = 200
pd.options.display.max_rows = 200
%matplotlib inline

from sklearn import preprocessing
import warnings; warnings.simplefilter('ignore')

In [10]:
# Sklearn stuff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.model_selection import ShuffleSplit, KFold

In [11]:
import time
import datetime
import pytz

In [12]:
def write_to_submission_file(predicted_labels, filename='logit_model'):
    df_submission = pd.DataFrame({'radiant_win_prob': predicted_labels}, 
                                     index=test_features.index)

    submission_filename = '{}_{}.csv'.format(filename,
        datetime.datetime.now(tz=pytz.timezone('US/Pacific')).strftime('%Y-%m-%d_%H-%M-%S'))
    
    df_submission.to_csv(submission_filename)
    
    print('Submission saved to {}'.format(submission_filename))

### Hero Lookup

In [36]:
# # # Getting Heroes ID according to the name
heroes = json.read('data2/odheroes.json')

AttributeError: module 'json' has no attribute 'read'

In [ ]:
for i in range (len(heroes['heroes'])):
    hero_id = heroes['heroes'][i]['id']
    hero_name = heroes['heroes'][i]['localized_name']
    heroes_dict[hero_id]= hero_name

In [29]:
def change_hero_name(row):
    if row in heroes_dict:
        return heroes_dict[row]
    
columns = [
    '1hero_id', 
    '2hero_id', 
    '3hero_id', 
    '4hero_id', 
    '5hero_id', 
    '6hero_id', 
    '7hero_id', 
    '8hero_id', 
    '9hero_id',
    '10hero_id',
    '11hero_id', 
    '12hero_id', 
    '13hero_id', 
    '14hero_id', 
    '15hero_id', 
    '16hero_id', 
    '17hero_id', 
    '18hero_id', 
    '19hero_id',
    '20hero_id',
    '21hero_id',
    '22hero_id'
    ]
    
for i in range(len(columns)):
    data[columns[i]] = data[columns[i]].apply(change_hero_name)

IndentationError: expected an indented block (<ipython-input-29-06c91f8c590c>, line 30)

## Logistic Regression 

Try doing logistic regression with just hot encoded heroes and see what results are like

In [13]:
SEED = 17

In [14]:
# Import Training and Test Data  
train_features = pd.read_csv('data2/dpc_train_features.csv', index_col='match_id')
train_targets = pd.read_csv('data2/dpc_train_target.csv', index_col='match_id')
test_features = pd.read_csv('data2/dpc_test_features.csv', index_col='match_id')
test_targets = pd.read_csv('data2/dpc_test_target.csv', index_col='match_id')

In [15]:
train_features.shape, train_targets.shape, test_features.shape, test_targets.shape

((1045, 66), (1045, 2), (117, 66), (117, 2))

In [17]:
df_full_features = pd.concat([train_features, test_features])

# Index to split the training and test data sets
idx_split = train_features.shape[0]

heroes_df = df_full_features[[f'{i}hero_id' for i in range(1, 23)]]

In [18]:
heroes_df.shape

(1162, 22)

In [19]:
# Quick check that hero ids are unique (i.e., no double picking)
# https://www.kaggle.com/kuzand/dota-2-winner-prediction-multilayer-nn-pytorch

np.all(df_full_features[[f'{i}hero_id' for i in range(1, 23)]].nunique(axis=1) == 22)

True

In [20]:
np.sort(np.unique(heroes_df[[f'{i}hero_id' for i in range(1, 23)]].values.flatten()))

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  25,  26,  27,
        28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,
        41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,
        54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,
        67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,  79,
        80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,  92,
        93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104, 105,
       106, 107, 108, 109, 110, 111, 112, 113, 114, 119, 120, 121, 129])

In [21]:
(unique, counts) = np.unique(heroes_df[[f'{i}hero_id' for i in range(1, 23)]],return_counts=True) 
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[  1 116]
 [  2 239]
 [  3 128]
 [  4 156]
 [  5 221]
 [  6  63]
 [  7 302]
 [  8 336]
 [  9 550]
 [ 10 466]
 [ 11  66]
 [ 12  64]
 [ 13 153]
 [ 14  29]
 [ 15 165]
 [ 16 398]
 [ 17 262]
 [ 18 339]
 [ 19 484]
 [ 20  22]
 [ 21  58]
 [ 22 108]
 [ 23 715]
 [ 25 507]
 [ 26  58]
 [ 27 202]
 [ 28  51]
 [ 29 209]
 [ 30  55]
 [ 31 119]
 [ 32  14]
 [ 33 304]
 [ 34  35]
 [ 35  82]
 [ 36  55]
 [ 37 266]
 [ 38 266]
 [ 39 162]
 [ 40  53]
 [ 41 554]
 [ 42 313]
 [ 43 260]
 [ 44  58]
 [ 45 139]
 [ 46 351]
 [ 47  63]
 [ 48  12]
 [ 49  68]
 [ 50  85]
 [ 51  56]
 [ 52 613]
 [ 53  65]
 [ 54 460]
 [ 55 228]
 [ 56  12]
 [ 57 484]
 [ 58 277]
 [ 59  70]
 [ 60 382]
 [ 61 196]
 [ 62  36]
 [ 63  34]
 [ 64 118]
 [ 65 341]
 [ 66 637]
 [ 67 115]
 [ 68 358]
 [ 69  81]
 [ 70  34]
 [ 71 264]
 [ 72 574]
 [ 73 362]
 [ 74 217]
 [ 75  78]
 [ 76 638]
 [ 77  78]
 [ 78  41]
 [ 79 635]
 [ 80  88]
 [ 81 128]
 [ 82 118]
 [ 83 172]
 [ 84 401]
 [ 85 277]
 [ 86 545]
 [ 87  76]
 [ 88 182]
 [ 89 327]
 [ 90 369]
 [ 91 276]
 [ 92  19]

In [22]:
heroes_df_ohe = heroes_df.copy()
for i in range(1, 23):
    heroes_df_ohe = pd.get_dummies(heroes_df_ohe, columns = [f'{i}hero_id'])
        
heroes_df_ohe.head()

X_heroes_train = heroes_df_ohe[:idx_split] # features set is only picks + dummies
X_heroes_test  = heroes_df_ohe[idx_split:]

In [23]:
print(f'Number of features: {heroes_df_ohe.shape[1]}')

Number of features: 2044


In [24]:
X_heroes_train.shape, X_heroes_test.shape

((1045, 2044), (117, 2044))

In [25]:
X_heroes_train.head()

,1hero_id_2,1hero_id_3,1hero_id_4,1hero_id_6,1hero_id_7,1hero_id_9,1hero_id_10,1hero_id_16,1hero_id_18,1hero_id_19,1hero_id_23,1hero_id_25,1hero_id_27,1hero_id_29,1hero_id_31,1hero_id_33,1hero_id_37,1hero_id_38,1hero_id_40,1hero_id_41,1hero_id_42,1hero_id_43,1hero_id_46,1hero_id_52,1hero_id_53,1hero_id_55,1hero_id_57,1hero_id_58,1hero_id_59,1hero_id_60,1hero_id_61,1hero_id_62,1hero_id_65,1hero_id_66,1hero_id_68,1hero_id_69,1hero_id_71,1hero_id_72,1hero_id_73,1hero_id_74,1hero_id_75,1hero_id_76,1hero_id_79,1hero_id_80,1hero_id_81,1hero_id_82,1hero_id_83,1hero_id_84,1hero_id_85,1hero_id_86,1hero_id_87,1hero_id_88,1hero_id_89,1hero_id_90,1hero_id_91,1hero_id_93,1hero_id_96,1hero_id_97,1hero_id_98,1hero_id_99,1hero_id_102,1hero_id_103,1hero_id_104,1hero_id_105,1hero_id_106,1hero_id_107,1hero_id_111,1hero_id_112,1hero_id_119,1hero_id_120,1hero_id_121,1hero_id_129,2hero_id_2,2hero_id_3,2hero_id_4,2hero_id_7,2hero_id_9,2hero_id_10,2hero_id_16,2hero_id_17,2hero_id_18,2hero_id_19,2hero_id_22,2hero_id_23,2hero_id_25,2hero_id_29,2hero_id_31,2hero_id_33,2hero_id_37,2hero_id_38,2hero_id_41,2hero_id_42,2hero_id_43,2hero_id_46,2hero_id_50,2hero_id_52,2hero_id_53,2hero_id_54,2hero_id_55,2hero_id_57,...,22hero_id_1,22hero_id_2,22hero_id_4,22hero_id_5,22hero_id_6,22hero_id_7,22hero_id_8,22hero_id_10,22hero_id_11,22hero_id_12,22hero_id_13,22hero_id_14,22hero_id_15,22hero_id_16,22hero_id_17,22hero_id_18,22hero_id_19,22hero_id_21,22hero_id_22,22hero_id_23,22hero_id_25,22hero_id_26,22hero_id_27,22hero_id_28,22hero_id_29,22hero_id_32,22hero_id_33,22hero_id_34,22hero_id_35,22hero_id_36,22hero_id_37,22hero_id_38,22hero_id_39,22hero_id_40,22hero_id_41,22hero_id_42,22hero_id_43,22hero_id_44,22hero_id_45,22hero_id_46,22hero_id_47,22hero_id_48,22hero_id_49,22hero_id_50,22hero_id_51,22hero_id_52,22hero_id_53,22hero_id_54,22hero_id_55,22hero_id_56,22hero_id_57,22hero_id_58,22hero_id_59,22hero_id_60,22hero_id_61,22hero_id_62,22hero_id_63,22hero_id_65,22hero_id_67,22hero_id_68,22hero_id_70,22hero_id_71,22hero_id_72,22hero_id_73,22hero_id_74,22hero_id_75,22hero_id_76,22hero_id_77,22hero_id_78,22hero_id_79,22hero_id_80,22hero_id_81,22hero_id_82,22hero_id_84,22hero_id_85,22hero_id_86,22hero_id_88,22hero_id_89,22hero_id_92,22hero_id_93,22hero_id_94,22hero_id_95,22hero_id_96,22hero_id_97,22hero_id_98,22hero_id_99,22hero_id_101,22hero_id_102,22hero_id_104,22hero_id_106,22hero_id_107,22hero_id_108,22hero_id_109,22hero_id_110,22hero_id_112,22hero_id_113,22hero_id_114,22hero_id_120,22hero_id_121,22hero_id_129
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5115031896,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4865407245,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4889341660,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5050335216,0,0,0,0,0,0,0,0,0,0,0

Hot encoding by getting dummies for each hero_id feature. 

In [26]:
#for i in range(1,23):
#        df_full_features = pd.get_dummies(df_full_features, columns = [f'{i}hero_id'])

In [27]:
df_full_features.shape # this should be same size as entire dataset

(1162, 66)

### Scaling 

Trying it out with minmax. From what I can tell, this seems like the most appropriate scaling method. 

In [ ]:
df_full_features_scaled = df_full_features.copy()
df_full_features_scaled[df_full_features.columns.tolist()] = MinMaxScaler().fit_transform(df_full_features_scaled[df_full_features.columns.tolist()])  # alternatively use StandardScaler

In [ ]:
df_full_features_scaled.head()

In [ ]:
df_full_features_scaled.shape

Okay, looks like we have the entire data set hot encoded and scaled (minmax). Time to split it back up and go forward. 

## Set targets, features, and verification set 

Should the combined train sets still have hero variables? did I do this right?

In [ ]:
# Encoded train/test features

X_train = df_full_features_scaled[:idx_split]
X_test = df_full_features_scaled[idx_split:]

y_train = train_targets.radiant_win
y_test = test_targets.radiant_win

In [ ]:
# keep full version of training set to use for final accuracy model 
X_train_full = X_train.copy() 
#y_train_full = y_train.copy()

In [ ]:
# Check to make sure everything is correct size
X_train.shape, X_train_full.shape, X_test.shape, y_train.shape, y_train_full.shape, y_test.shape

Everything seems in order, let's do our validation split and get into the modelliing. 

### Train Validation Split

Train Validation Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.1, random_state = SEED) ## see top, SEED = 17

In [ ]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
print(f'Shape of test set: {X_test.shape}')
print(f'Shape of validation set: {X_valid.shape}')
print(f'Shape of train set: {X_train.shape}')

## Logistic Regression

To summarize what we're working with. The data has been hot encoded (pd.dummy) and scaled (minmax). After that, we performed another split on the training data to create a validation set for the purpose of hyperparameter tuning.  

### Grid Search 1. 



In [ ]:
## Source: https://www.kaggle.com/kuzand/bag-of-heroes-logistic-regression#notebook-container

def logit_cv(X_train, y_train, cv=5, random_state=SEED):
    
    logit = LogisticRegression(random_state=SEED, solver='liblinear')

    c_values = np.logspace(-2, 1, 20)

    logit_grid_searcher = GridSearchCV(estimator=logit, param_grid={'C': c_values},
                                       scoring='roc_auc',return_train_score=True, cv=cv,
                                       n_jobs=-1, verbose=0)

    logit_grid_searcher.fit(X_train, y_train)
    
    cv_scores = []
    for i in range(logit_grid_searcher.n_splits_):
        cv_scores.append(logit_grid_searcher.cv_results_[f'split{i}_test_score'][logit_grid_searcher.best_index_])
    print(f'CV scores: {cv_scores}\n')
    print(f'Mean: {np.mean(cv_scores)}, std: {np.std(cv_scores)}\n')
    
    return logit_grid_searcher.best_estimator_, np.array(cv_scores) 

In [ ]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

In [ ]:
#logit = logit_cv(X_train, y_train, cv=skf, random_state=SEED)[0]

Computing Logit model cv scores 

In [ ]:
%%time
# Computing logit cv scores

logit_0, cv_scores_0 = logit_cv(X_train, y_train, cv=skf, random_state=SEED)

In [ ]:
logit_0
print(f"Accuracy Score on Train Set: {logit_0.score(X_train, y_train)}")
print(f"Predict Probability on Train Set: {logit_0.score(X_train, y_train)}")
print(f'Log Regression validation roc_auc score: {roc_auc_score(y_pred, y_valid)}')

In [ ]:
logit.fit(X_train, y_train)
y_pred = logit.predict(X_valid)
y_pred_proba = logit.predict_proba(X_valid)

print(f"Accuracy Score on Train Set: {logit.score(X_train, y_train)}")
print(f"Predict Probability on Train Set: {logit.score(X_train, y_train)}")
print(f'Log Regression validation roc_auc score: {roc_auc_score(y_pred, y_valid)}')

In [ ]:
logit.fit(X_train, y_train)

logit_pred = logit.predict_proba(X_valid)

write_to_submission_file(logit_pred, filename='logit_model')

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_valid, y_pred)

In [ ]:
%%time
# https://www.kaggle.com/smsrikanthreddy/dota-2-win-prediction-logistic-regression

#calcuate ROC-AUC for each split
#logistic Regression
C = 0.01
penalty = 'l2'
max_iter = 100
solver = 'lbfgs'
random_state = 17
n_jobs = -1
verbose = 0

clf_lr_1 = LogisticRegression(C=C,
                            penalty=penalty,
                            max_iter=max_iter, 
                            random_state=random_state,
                            verbose=verbose,
                            n_jobs=n_jobs,
                           solver=solver)

#cv_scores_lr1 = cross_val_score(clf_lr_1, X_train, y, cv=cv, scoring='roc_auc')

In [ ]:
%%time
clf_lr_1.fit(X_train, y_train)
y_pred = clf_lr_1.predict(X_valid)
print('Log Regression validation roc_auc score {} '.format(roc_auc_score(y_pred, y_valid)))

In [ ]:
#logit.fit(X_train, y_train)

y_pred = logit.predict(X_valid)

logit_pred = logit.predict_proba(X_valid)

#write_to_submission_file(logit_pred, filename='logit_hero_roles')

In [ ]:
#print(f"Accuracy Score on Train Set: {logit.score(X_train, y_train)}")
#print(f'Log Regression validation roc_auc score: {roc_auc_score(y_pred, y_valid)}')

In [ ]:
#logistic Regression

C = 0.01
penalty = 'l2'
max_iter = 100
solver = 'lbfgs'
random_state = 17
n_jobs = -1
verbose = 1

dpc_log = LogisticRegression(C=C,
                            penalty=penalty,
                            max_iter=max_iter, 
                            random_state=random_state,
                            verbose=verbose,
                            n_jobs=n_jobs,
                           solver=solver)

In [ ]:
%%time

dpc_log.fit(X_train, y_train)
y_pred = dpc_log.predict(X_valid)

print(f"Accuracy Score on Train Set: {dpc_log.score(X_train, y_train)}")

print(f'Log Regression validation roc_auc score: {roc_auc_score(y_pred, y_valid)}')

In [ ]:
plt.figure()
plt.plot(C_range, cross_validation_scores,label="Cross Validation Score",marker='.')
plt.legend()
plt.xscale("log")
plt.xlabel('Regularization Parameter: C')
plt.ylabel('Cross Validation Score')
plt.grid()
plt.show();

which_max = np.array(cross_validation_scores).argmax()

print("The best model has C = ",C_range[which_max])

In [ ]:
y_pred_scaled = dpc_log_scaled.predict(X_valid1)
print('Log Regression validation roc_auc score {} '.format(roc_auc_score(y_pred, y_valid)))
print(dpc_log.score(X_valid,y_valid))
print(dpc_log_scaled.score(X_scaled_valid,y_valid))

In [ ]:
from sklearn.linear_model import LogisticRegression

validation_scores = []
train_scores = []

C_range = np.array([.00000001,.0000001,.000001,.00001,.0001,.001,0.1,\
                1,10,100,1000,10000,100000,1000000,10000000,100000000,1000000000])

for c in C_range:
    logreg = LogisticRegression(C = c,random_state=17)
    logreg.fit(X_train,y_train)
    
    # train on traning set
    train_scores.append(logreg.score(X_train,y_train))
    # score on validation set
    validation_scores.append(logreg.score(X_valid,y_valid))

In [ ]:
#Store the results
cross_validation_scores = []

C_range = np.array([.00000001,.0000001,.000001,.00001,.0001,.001,.1,\
                1,10,100,1000,10000,100000,1000000,10000000,100000000,1000000000])

#Do some cross validation
for c in C_range:
    LR_model = LogisticRegression(C=c,random_state=17)
    
    # the cross validation score (mean of scores from all folds)
    cv_score = np.mean(cross_val_score(LR_model, X_remainder, y_remainder, cv = 5))
    
    cross_validation_scores.append(cv_score)
    

K-FOLD CROSS VALIDATION

GRID SEARCH

Attempt #2

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

C_range = np.logspace(-2, 1, 20)

grid={"C":C_range, "penalty":["l1","l2"]}# l1 lasso l2 ridge
dpc_log = LogisticRegression(random_state=17)
dpc_logcv =GridSearchCV(dpc_log,grid,cv=10)
dpc_logcv.fit(X_train,y_train)

print("tuned hyperparameters :(best parameters) ", dpc_logcv.best_params_)
print("accuracy :",dpc_logcv.best_score_)

In [ ]:
# Instantiate model 

scores = cross_val_score(dpc_log, X_train, y_train, cv = 5)

# Every score corresponds to a fold
for i in range(len(scores)) :
    print(f"Fold {i+1}: {scores[i]}")
    
print(f"Average Score:{np.mean(scores)}")

In [ ]:
# Create regularization penalty space
#penalty = ['l1', 'l2']

# Create regularization hyperparameter space
#C = np.logspace(0, 4, 10)

# Create hyperparameter options
#hyperparameters = dict(C=C, penalty=penalty)

In [ ]:
# Create grid search using 5-fold cross validation
clf = GridSearchCV(logit, hyperparameters, cv=5, verbose=0)

In [ ]:
#Store the results
cross_validation_scores = []

C_range = np.array([.00000001,.0000001,.000001,.00001,.0001,.001,.1,\
                1,10,100,1000,10000,100000,1000000,10000000,100000000,1000000000])

#Do some cross validation
for c in C_range:
    dpc_log = LogisticRegression(C=c,random_state=SEED, solver = 'liblinear')
    cv_score = np.mean(cross_val_score(dpc_log, X_train1, y_train, cv = 5))
    cross_validation_scores.append(cv_score)

plt.figure()
plt.plot(C_range, cross_validation_scores,label="Cross Validation Score",marker='.')
plt.legend()
plt.xscale("log")
plt.xlabel('Regularization Parameter: C')
plt.ylabel('Cross Validation Score')
plt.grid()
plt.show();

which_max = np.array(cross_validation_scores).argmax()

print("The best model has C = ",C_range[which_max])

Trying once with logisticsCV version

In [ ]:
# Try once with logistic regression with cross validation built in 

dpc_lrcv = LogisticRegressionCV(
    Cs = 10,
    fit_intercept=True,
    cv='warn',
    dual=False,
    penalty='l2',
    scoring=None,
    solver='lbfgs',
    tol=0.0001,
    max_iter=100,
    class_weight=None,
    n_jobs=None,
    verbose=0,
    refit=True,
    intercept_scaling=1.0,
    multi_class='warn',
    random_state= 17,
    l1_ratios=None
)

dpc_lrcv.fit(X_train, y_train)

In [ ]:
y_pred = clf_lr.predict(X_valid)
print(f"Accuracy Score on Train Set: {dpc_lrcv.score(X_train,y_train)}")
print('Log Regression validation roc_auc score {} '.format(roc_auc_score(y_pred, y_valid)))